In [5]:
import os
import re
import helpers
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import HashingVectorizer
from datetime import datetime

# ------------- read in entries from all files -------------

data_raw = []
data_raw_filtered = []
data_directory = "data"

for file in os.listdir("data"):
    if file.endswith(".json"):
        data = helpers.read_file(data_directory + "/" + file)
        data_raw.append(data)

        data_raw_filtered = []
for file in data_raw:
    for entry in file:
        if ("topics" in entry) and ("body" in entry):
            data_raw_filtered.append(entry)

# ------------------ create bag of words -----------------

data_clean = []
topics_has_earn_word = []

for entry in data_raw_filtered:
    words_list = re.sub("[^a-zA-Z]", " ", entry['body'])
    data_clean.append([w for w in words_list.lower().split()])
    if "earn" in entry["topics"]:
        topics_has_earn_word.append(1)
    else:
        topics_has_earn_word.append(0)

data_clean_train = []

for line in data_clean:
    data_clean_train.append(" ".join(line))  # glue all words together into a list of strings


# ------------------  'normal' BoW -----------------

# --- from hear I change a little the first I chose just 5 articles and the first 40 characters to see how my apraoch works

vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None
                             )
newdata=[]
for i in data_clean_train[:5]:
    newdata.append(i[:40])
train_data_features = vectorizer.fit_transform(newdata)

train_data_features_array = train_data_features.toarray()
# print train_data_features_array.shape

shingles=vectorizer.get_feature_names()
print len(vectorizer.get_feature_names())


26


In [6]:
# hear I get a list of the 5 articles and  words from vectorizer
print vectorizer.get_feature_names()
print train_data_features_array

[u'agriculture', u'argentine', u'board', u'champion', u'computer', u'continued', u'crop', u'department', u'figures', u'grain', u'ha', u'in', u'inc', u'it', u'its', u'of', u'products', u'reported', u'said', u'show', u'showers', u'systems', u'terminal', u'the', u'throughout', u'week']
[[0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1]
 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 1 1 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0]]


In [14]:
# Array form the previus
minhashArray=train_data_features_array

# get the row and columns of arrray
r=np.shape(minhashArray)[0]
c=np.shape(minhashArray)[1]
#Result=np.zeros((r,c))

# Creating the phi size for permutation
phiSize=r
permutationArray=np.array(range(phiSize))
np.random.shuffle(permutationArray)
#----------------------------------------

Result=np.zeros((r,c))
# for creat result I iterate trough rows and each time I shuffle permutationArrya and base of the array I create
# a result which obtain base of the first occurance of the 1  (I checked with three articles and not changing permutationArray
# and the result was reasonable)
for rMinhash in range(r):
    np.random.shuffle(permutationArray)
    for Col in range(c):
        placeFinder=0
        for Row in permutationArray:
            placeFinder+=1
            if minhashArray[Row][Col]==1:
                Result[rMinhash][Col]=int(placeFinder)

# I show the result her to see how the matrix look like after permutation
print Result

# now I make a bukket list with the similarity in thier columns
bucket=[]
bucketList=[]
WordColNr1=-1
listOfTheWords=vectorizer.get_feature_names()
for word in listOfTheWords:
    WordColNr1+=1
    WordColNr2=-1
    bucket=[]
    for word in listOfTheWords:
        WordColNr2+=1
        if np.array_equal(Result[:,WordColNr1],Result[:,WordColNr2]):
            bucket.append(word)
        
    if bucket in bucketList:
        bucketList.remove(bucket)
    bucketList.append(bucket)
        
print "\n\nBuckets:\n\n"
for article in bucketList:
    print article,"\n\n"
        

[[ 5.  3.  3.  1.  4.  2.  3.  5.  3.  3.  4.  2.  4.  4.  1.  1.  1.  5.
   4.  3.  2.  4.  4.  5.  2.  2.]
 [ 5.  1.  3.  3.  4.  2.  1.  5.  1.  1.  4.  2.  4.  4.  3.  3.  3.  5.
   4.  1.  2.  4.  4.  5.  2.  2.]
 [ 5.  1.  4.  4.  3.  2.  1.  5.  1.  1.  3.  2.  4.  3.  4.  4.  4.  5.
   4.  1.  2.  3.  3.  5.  2.  2.]
 [ 4.  5.  5.  1.  2.  3.  5.  4.  5.  5.  2.  3.  2.  2.  1.  1.  1.  4.
   2.  5.  3.  2.  2.  4.  3.  3.]
 [ 3.  4.  4.  2.  1.  5.  4.  3.  4.  4.  1.  5.  2.  1.  2.  2.  2.  3.
   2.  4.  5.  1.  1.  5.  5.  5.]]


Buckets:


[u'board'] 


[u'champion', u'its', u'of', u'products'] 


[u'agriculture', u'department', u'reported'] 


[u'inc', u'said'] 


[u'argentine', u'crop', u'figures', u'grain', u'show'] 


[u'computer', u'ha', u'it', u'systems', u'terminal'] 


[u'the'] 


[u'continued', u'in', u'showers', u'throughout', u'week'] 


